## Estimating Accuracy from Unlabeled Data

Try to implement the Bayesian Error Estimation (BEE) model in following paper. This is based on my limited understanding and I can't guarantee the implementation is bug-free.

[Emmanouil Antonios Platanios, Avinava Dubey, Tom Mitchell ; Proceedings of The 33rd International Conference on Machine Learning, PMLR 48:1416-1425, 2016.](http://proceedings.mlr.press/v48/platanios16.html)

In [24]:
import numpy as np

### 0. Generate dummy data

In real-life, the estimations should come from estimators

In [161]:
num_iters = 50
num_estimators = 4
num_samples = 1000

In [162]:
labeling_matrix = np.random.randint(0, 2, (num_samples, num_estimators))

### 1. Gibbs Sampling

In [163]:
true_labels = np.random.randint(0, 2, num_samples)
error_rates = 0.2*np.random.random(num_estimators)
print("initial error rate:", error_rates)

initial error rate: [0.17222602 0.0660327  0.1914704  0.00341093]


In [164]:
# set the hyper-parameters > 1 so that it's convex shape with mean 0.5. 
alpha_p, beta_p, alpha_e, beta_e = 2, 2, 2, 10

In [157]:
def sample_p():
    """ equation 2 + discounting the old label when sampling
    """
    sigma_l = np.sum(true_labels)
    return np.random.beta(alpha_p + sigma_l, beta_p + num_samples - sigma_l)

In [158]:
def sample_l(p, i):
    """ equation 3
    """
    pi = 1
    # the number of correct predictions of each estimator. dim [num_estimators, 1]
    pi = np.zeros(2)  # the pi value for l=0 and l=1
    for k in range(2):    
        num_corrects = labeling_matrix[i,:] == k
        temp = np.power(error_rates, 1 - num_corrects)*np.power(1 - error_rates, num_corrects)
        pi[k] = np.prod(temp)
    prob = pi * np.asarray([1-p, p])
    positive_prob = prob[1]/np.sum(prob)
    return random.binomial(1, positive_prob)

In [159]:
def sample_e(j):
    """ equation 4
    """
    sigma_j = np.sum(labeling_matrix[:, j] == true_labels)
    return np.random.beta(alpha_e + sigma_j, beta_e + num_samples - sigma_j)

In [167]:
for it in range(num_iters):
    for i in range(num_samples):
        p = sample_p()
        true_labels[i] = sample_l(p, i)
    for j in range(num_estimators):
        error_rates[j] = sample_e(j)
    print("Iteration", it, ":")
    print("Error rates", error_rates)
    #print(true_labels)

Iteration 0 :
Error rates [0.49919948 0.50076524 0.49701213 0.51337747]
Iteration 1 :
Error rates [0.48429592 0.46654233 0.500574   0.49084223]
Iteration 2 :
Error rates [0.49538885 0.52638187 0.51562536 0.55249207]
Iteration 3 :
Error rates [0.50461455 0.49896605 0.45405736 0.43513692]
Iteration 4 :
Error rates [0.50585267 0.50016561 0.54600378 0.56159173]
Iteration 5 :
Error rates [0.46981574 0.54323903 0.48107222 0.47180822]
Iteration 6 :
Error rates [0.52413538 0.46328766 0.48120521 0.5242842 ]
Iteration 7 :
Error rates [0.47433243 0.53387303 0.50810609 0.50066616]
Iteration 8 :
Error rates [0.52163844 0.46620497 0.48457315 0.48836471]
Iteration 9 :
Error rates [0.4845304  0.53178741 0.52696287 0.51529558]
Iteration 10 :
Error rates [0.54335754 0.45901428 0.50354554 0.47233309]
Iteration 11 :
Error rates [0.46115397 0.51916769 0.47709521 0.51915642]
Iteration 12 :
Error rates [0.56661273 0.48283402 0.52409068 0.47013   ]
Iteration 13 :
Error rates [0.42143031 0.53541938 0.50889627 